In [1]:
import yfinance as yf, pandas as pd, shutil, os, time, glob, smtplib, ssl
from datetime import datetime
from get_all_tickers import get_tickers as gt

In [12]:
target_date = "2019-01-22"
num_days_analyze = 10
num_days_since_begin_prev_week = 5

In [13]:
# OBV Analysis, feel free to replace this section with your own analysis -------------------------------------------------------------------------
list_files = (glob.glob("/Users/kyletopasna/Documents/robinhood/Daily_Stock_Report/Stocks/*.csv")) # Creates a list of all csv filenames in the stocks folder
new_data = [] #  This will be a 2D array to hold our stock name and OBV score
interval = 0  # Used for iteration
while interval < len(list_files):
    file = list_files[interval]
    #print(file)
    
    Data = pd.read_csv(file)# Gets the last 10 days of trading for the current stock in iteration
    if len(Data) < num_days_analyze:
        interval += 1
    else:
        if len(Data.loc[Data['Date']==target_date].index) == 0:
            #print(Data.tail(1))
            interval +=1
        else:
            #print(Data.loc[Data['Date']==target_date].index)
            start_date_idx = (Data.loc[Data['Date']==target_date].index - num_days_analyze)[0]
            end_date_idx = (Data.loc[Data['Date']==target_date].index)[0]
            Data = Data.iloc[start_date_idx:end_date_idx]
            if len(Data) < num_days_analyze:
                interval +=1
            else:
                pos_move = []  # List of days that the stock price increased
                neg_move = []  # List of days that the stock price increased
                OBV_Value = 0  # Sets the initial OBV_Value to zero
                count = 0
                while (count < 10== num_days_analyze):  # 10 because we are looking at the last 10 trading days
                    if Data.iloc[count,1] < Data.iloc[count,4]:  # True if the stock increased in price
                        pos_move.append(count)  # Add the day to the pos_move list
                    elif Data.iloc[count,1] > Data.iloc[count,4]:  # True if the stock decreased in price
                        neg_move.append(count)  # Add the day to the neg_move list
                    count += 1
                count2 = 0
                for i in pos_move:  # Adds the volumes of positive days to OBV_Value, divide by opening price to normalize across all stocks
                    OBV_Value = round(OBV_Value + (Data.iloc[i,5]/Data.iloc[i,1]))
                for i in neg_move:  # Subtracts the volumes of negative days from OBV_Value, divide by opening price to normalize across all stocks
                    OBV_Value = round(OBV_Value - (Data.iloc[i,5]/Data.iloc[i,1]))
                Stock_Name = ((os.path.basename(list_files[interval])).split(".csv")[0])  # Get the name of the current stock we are analyzing
                new_data.append([Stock_Name, OBV_Value])  # Add the stock name and OBV value to the new_data list
                #print(new_data[-1])
                interval += 1

In [14]:
df = pd.DataFrame(new_data, columns = ['Stock', 'OBV_Value'])  # Creates a new dataframe from the new_data list
df["Stocks_Ranked"] = df["OBV_Value"].rank(ascending = False)  # Rank the stocks by their OBV_Values
df.sort_values("OBV_Value", inplace = True, ascending = False)  # Sort the ranked stocks
df.to_csv("/Users/kyletopasna/Documents/robinhood/Daily_Stock_Report/Targeted_OBV_Ranked.csv", index = False)  # Save the dataframe to a csv without the index column
# OBV_Ranked.csv now contains the ranked stocks that we want recalculate daily and receive in a digestable format.

# Code to email yourself your anaysis -----------------------------------------------------------------------------------
Analysis = pd.read_csv("/Users/kyletopasna/Documents/robinhood/Daily_Stock_Report/Targeted_OBV_Ranked.csv")  # Read in the ranked stocks

In [15]:
top_10 = Analysis['Stock'].head(5)
list_files = []
for stock in top_10:
    list_files.append("/Users/kyletopasna/Documents/robinhood/Daily_Stock_Report/Stocks/" + stock + ".csv")
print(list_files)

['/Users/kyletopasna/Documents/robinhood/Daily_Stock_Report/Stocks/LYG.csv', '/Users/kyletopasna/Documents/robinhood/Daily_Stock_Report/Stocks/ABEV.csv', '/Users/kyletopasna/Documents/robinhood/Daily_Stock_Report/Stocks/BAC.csv', '/Users/kyletopasna/Documents/robinhood/Daily_Stock_Report/Stocks/GE.csv', '/Users/kyletopasna/Documents/robinhood/Daily_Stock_Report/Stocks/SWN.csv']


In [16]:
interval = 0  # Used for iteration
composite_fund = []
while interval < len(list_files):
    file = list_files[interval]
    Data = pd.read_csv(file)
    start_date_idx = (Data.loc[Data['Date']==target_date].index - num_days_since_begin_prev_week)[0]
    end_date_idx = (Data.loc[Data['Date']==target_date].index)[0]
    Data = Data.iloc[start_date_idx:end_date_idx+1]
    print(Data)
    open_price = Data.iloc[0,1]
    close_price = Data.iloc[-1,1]
    stock_name = ((os.path.basename(list_files[interval])).split(".csv")[0])
    percent_change = ((close_price - open_price) / open_price) * 100
    composite_fund.append([stock_name, percent_change])
    interval +=1

            Date      Open      High       Low     Close    Volume  Dividends  \
4310  2019-01-14  2.425504  2.539336  2.416747  2.504311  16730700        0.0   
4311  2019-01-15  2.478042  2.486798  2.416748  2.478042  13479400        0.0   
4312  2019-01-16  2.495554  2.539336  2.495554  2.513067  15687100        0.0   
4313  2019-01-17  2.513067  2.548093  2.504311  2.530580   8546200        0.0   
4314  2019-01-18  2.591874  2.618143  2.574362  2.600631   8695400        0.0   
4315  2019-01-22  2.574362  2.591874  2.565605  2.583118   8450200        0.0   

      Stock Splits  
4310           0.0  
4311           0.0  
4312           0.0  
4313           0.0  
4314           0.0  
4315           0.0  
            Date      Open      High       Low     Close    Volume  Dividends  \
5501  2019-01-14  4.405390  4.443205  4.367576  4.414844  34416900        0.0   
5502  2019-01-15  4.443205  4.490473  4.433751  4.471566  41402300        0.0   
5503  2019-01-16  4.452658  4.499927  4.40

In [17]:
composite_fund=pd.DataFrame(composite_fund, columns = ['Stock', 'Pct_Change'])
composite_fund['Pct_Change'].mean()

5.481020127310072

In [41]:
composite_fund

,Stock,Pct_Change
